## Zadanie 2

In [0]:
actorsPath = "dbfs:/FileStore/tables/Files/actors.csv"
actorsDf = spark.read.format("csv").option("header","true").option("inferSchema","true").load(actorsPath)
display(actorsDf.limit(20))

imdb_title_id,ordering,imdb_name_id,category,job,characters
tt0000009,1,nm0063086,actress,null,[Miss Geraldine Holbrook (Miss Jerry)]
tt0000009,2,nm0183823,actor,null,[Mr. Hamilton]
tt0000009,3,nm1309758,actor,null,[Chauncey Depew - the Director of the New York Central Railroad]
tt0000009,4,nm0085156,director,null,null
tt0000574,1,nm0846887,actress,null,[Kate Kelly]
tt0000574,2,nm0846894,actor,null,[School Master]
tt0000574,3,nm3002376,actor,null,[Steve Hart]
tt0000574,4,nm0170118,actress,null,null
tt0000574,5,nm0846879,director,null,null
tt0000574,6,nm0317210,producer,producer,null


In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
actors_schema = StructType([
    StructField("imdb_title_id", StringType(), True),  
    StructField("ordering", IntegerType(), True),  
    StructField("imdb_name_id", StringType(), True),  
    StructField("category", StringType(), True),  
    StructField("job", StringType(), True),
    StructField("characters", StringType(), True)
])

In [0]:
actorsDfSchema = spark.read.format("csv").option("header", "true").schema(actors_schema).load(actorsPath)
display(actorsDfSchema.limit(20))

imdb_title_id,ordering,imdb_name_id,category,job,characters
tt0000009,1,nm0063086,actress,null,[Miss Geraldine Holbrook (Miss Jerry)]
tt0000009,2,nm0183823,actor,null,[Mr. Hamilton]
tt0000009,3,nm1309758,actor,null,[Chauncey Depew - the Director of the New York Central Railroad]
tt0000009,4,nm0085156,director,null,null
tt0000574,1,nm0846887,actress,null,[Kate Kelly]
tt0000574,2,nm0846894,actor,null,[School Master]
tt0000574,3,nm3002376,actor,null,[Steve Hart]
tt0000574,4,nm0170118,actress,null,null
tt0000574,5,nm0846879,director,null,null
tt0000574,6,nm0317210,producer,producer,null


## Zadanie 3

In [0]:
# Wczytywanie na poprawnych plikach
actorsPermissive = spark.read.format('csv').option("header", "true").schema(actors_schema).option("mode", "PERMISSIVE").load(actorsPath)
actorsDropmalformed = spark.read.format('csv').option("header", "true").schema(actors_schema).option("mode", "DROPMALFORMED").load(actorsPath)
actorsFailfast = spark.read.format('csv').option("header", "true").schema(actors_schema).option("mode", "FAILFAST").load(actorsPath)
actorsBadRecords = spark.read.format("csv").option("header", "true").schema(actors_schema).option("badRecordsPath", "/mnt/source/badrecords")

In [0]:
from pyspark.sql.functions import *
# zepsucie plików
actorsPermissive = actorsPermissive.withColumn("ordering", when(col("ordering") > 7, 'nan').otherwise(col("ordering")))
actorsPermissive.write.mode("overwrite").csv("dbfs:/tmp/zeputy.csv", header=True)

In [0]:
crashedDfPermissive = spark.read.format('csv').option("header", "true").schema(actors_schema).option("mode", "PERMISSIVE").load("dbfs:/tmp/zeputy.csv")
display(crashedDfPermissive.limit(20)) # wartości null w kolumnie "ordering"

imdb_title_id,ordering,imdb_name_id,category,job,characters
tt0000009,1,nm0063086,actress,null,[Miss Geraldine Holbrook (Miss Jerry)]
tt0000009,2,nm0183823,actor,null,[Mr. Hamilton]
tt0000009,3,nm1309758,actor,null,[Chauncey Depew - the Director of the New York Central Railroad]
tt0000009,4,nm0085156,director,null,null
tt0000574,1,nm0846887,actress,null,[Kate Kelly]
tt0000574,2,nm0846894,actor,null,[School Master]
tt0000574,3,nm3002376,actor,null,[Steve Hart]
tt0000574,4,nm0170118,actress,null,null
tt0000574,5,nm0846879,director,null,null
tt0000574,6,nm0317210,producer,producer,null


In [0]:
crashedDfDropmalformed = spark.read.format('csv').option("header", "true").schema(actors_schema).option("mode", "DROPMALFORMED").load("dbfs:/tmp/zeputy.csv")
display(crashedDfDropmalformed.limit(20)) # usunął wiersze z null w kolumnie "ordering"

imdb_title_id,ordering,imdb_name_id,category,job,characters
tt0000009,1,nm0063086,actress,null,[Miss Geraldine Holbrook (Miss Jerry)]
tt0000009,2,nm0183823,actor,null,[Mr. Hamilton]
tt0000009,3,nm1309758,actor,null,[Chauncey Depew - the Director of the New York Central Railroad]
tt0000009,4,nm0085156,director,null,null
tt0000574,1,nm0846887,actress,null,[Kate Kelly]
tt0000574,2,nm0846894,actor,null,[School Master]
tt0000574,3,nm3002376,actor,null,[Steve Hart]
tt0000574,4,nm0170118,actress,null,null
tt0000574,5,nm0846879,director,null,null
tt0000574,6,nm0317210,producer,producer,null


In [0]:
crashedDfFailfast = spark.read.format('csv').option("header", "true").schema(actors_schema).option("mode", "FAILFAST").load("dbfs:/tmp/zeputy.csv")
display(crashedDfFailfast.limit(20)) # przerwał operację, gdy natrafił na błąd

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 64.0 failed 1 times, most recent failure: Lost task 0.0 in stage 64.0 (TID 173) (ip-10-172-190-0.us-west-2.compute.internal executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/tmp/zeputy.csv/part-00000-tid-7531742791831032783-427bce3c-93b4-411b-9c8c-eb8bee5263a1-163-1-c000.csv.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:704)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:673)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$hasNext$1(FileScanRDD.scala:493)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:486)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:82)
	at org.apache.spark.sql.execution.collect.Collector.$anonfun$processFunc$1(Collector.scala:208)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:179)
	at org.apache.spark.scheduler.Task.$anonfun$run$5(Task.scala:142)
	at com.databricks.unity.EmptyHandle$.runWithAndClose(UCSHandle.scala:126)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:142)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:97)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$13(Executor.scala:904)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1741)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:907)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:761)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
Caused by: org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1936)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:103)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:507)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at org.apache.spark.util.CompletionIterator.hasNext(CompletionIterator.scala:31)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:619)
	... 28 more
Caused by: org.apache.spark.sql.catalyst.util.BadRecordException: java.lang.NumberFormatException: For input string: "nan"
Caused by: java.lang.NumberFormatException: For input strin

In [0]:
crashedDfBadRecords = spark.read.format("csv").option("header", "true").schema(actors_schema).option("badRecordsPath", "/mnt/source/badrecords").load("dbfs:/tmp/zeputy.csv")
display(crashedDfBadRecords.limit(20)) # usunął wiersze z null w kolumnie "ordering"

imdb_title_id,ordering,imdb_name_id,category,job,characters
tt0000009,1,nm0063086,actress,null,[Miss Geraldine Holbrook (Miss Jerry)]
tt0000009,2,nm0183823,actor,null,[Mr. Hamilton]
tt0000009,3,nm1309758,actor,null,[Chauncey Depew - the Director of the New York Central Railroad]
tt0000009,4,nm0085156,director,null,null
tt0000574,1,nm0846887,actress,null,[Kate Kelly]
tt0000574,2,nm0846894,actor,null,[School Master]
tt0000574,3,nm3002376,actor,null,[Steve Hart]
tt0000574,4,nm0170118,actress,null,null
tt0000574,5,nm0846879,director,null,null
tt0000574,6,nm0317210,producer,producer,null


## Zadanie 4

In [0]:
actorsDf.write.option("header", "true").format("parquet").mode("overwrite").save("dbfs:/tmp/actors.parquet")
actorsParquet = spark.read.format("parquet").schema(actors_schema).option("header", "true").load("dbfs:/tmp/actors.parquet")
display(actorsParquet.limit(20))

imdb_title_id,ordering,imdb_name_id,category,job,characters
tt3249124,4,nm6005417,actor,null,[Tom]
tt3249124,5,nm1871431,director,null,null
tt3249124,6,nm1862032,producer,producer,null
tt3249124,7,nm4261282,composer,null,null
tt3249124,8,nm1677303,production_designer,null,null
tt3249158,1,nm2946712,actor,null,[Sebastian]
tt3249158,2,nm2578315,actress,null,[Miranda]
tt3249158,3,nm7033017,actor,null,[Jacinto]
tt3249158,4,nm7033021,actor,null,[Rosa]
tt3249158,5,nm1848095,director,null,null


In [0]:
actorsDf.write.option("header", "true").format("json").mode("overwrite").save("dbfs:/tmp/actors.json")
actorsJson = spark.read.format("json").schema(actors_schema).option("header", "true").load("dbfs:/tmp/actors.json")
display(actorsJson.limit(20))

imdb_title_id,ordering,imdb_name_id,category,job,characters
tt3249124,4,nm6005417,actor,null,[Tom]
tt3249124,5,nm1871431,director,null,null
tt3249124,6,nm1862032,producer,producer,null
tt3249124,7,nm4261282,composer,null,null
tt3249124,8,nm1677303,production_designer,null,null
tt3249158,1,nm2946712,actor,null,[Sebastian]
tt3249158,2,nm2578315,actress,null,[Miranda]
tt3249158,3,nm7033017,actor,null,[Jacinto]
tt3249158,4,nm7033021,actor,null,[Rosa]
tt3249158,5,nm1848095,director,null,null
